<a href="https://colab.research.google.com/github/fwangliberty/AIoTDesign-Frontend/blob/master/cnn_small2_cicids2018_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection by using small CICIDS 2018 DataSet with 4000 examples each type

In [1]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

## Step 1. Read cleaned CICIDS2018 small dataset

Connect to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
# load train data
cleanfile = '/content/drive/My Drive/CICIDS2018/xtrain_small2.csv'
labelfile = '/content/drive/My Drive/CICIDS2018/ytrain_small2.csv'
xtestsmall = '/content/drive/My Drive/CICIDS2018/xtest_small2.csv'
ytestsmall = '/content/drive/My Drive/CICIDS2018/ytest_small2.csv'
SEED=2

In [47]:
X_train = pd.read_csv(cleanfile, skiprows=0,index_col=0) 
y_train = pd.read_csv(labelfile, skiprows=0,index_col=0)  
X_test = pd.read_csv(xtestsmall, skiprows=0, index_col=0) 
y_test = pd.read_csv(ytestsmall, skiprows=0,index_col=0) 
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77
0,443.0,6.0,642.0,3.0,0.0,77.0,0.0,46.0,0.0,25.666667,23.459184,0.0,0.0,0.0,0.0,119937.694700,4672.897196,3.210000e+02,4.270925e+02,623.0,19.0,642.0,3.210000e+02,4.270925e+02,623.0,19.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1.0,0.0,0.0,0.0,60.0,0.0,4672.897196,0.000000,0.0,46.0,30.750000,21.685248,470.250000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,41.000000,25.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,77.0,0.0,0.0,256.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,80.0,6.0,932.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,2145.922747,9.320000e+02,0.000000e+00,932.0,932.0,932.0,9.320000e+02,0.000000e+00,932.0,932.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,2145.922747,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,225.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,80.0,6.0,126.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,15873.015873,1.260000e+02,0.000000e+00,126.0,126.0,126.0,1.260000e+02,0.000000e+00,126.0,126.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,15873.015873,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2047.0,-1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,80.0,17.0,119624671.0,108107.0,0.0,3459424.0,0.0,32.0,32.0,32.000000,0.000000,0.0,0.0,0.0,0.0,28918.984450,903.718264,1.106550e+03,2.145599e+04,1174039.0,0.0,120000000.0,1.106550e+03,2.145599e+04,1174039.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,864856.0,0.0,903.718264,0.000000,32.0,32.0,32.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.000296,32.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108107.0,3459424.0,0.0,0.0,-1.0,-1.0,108106.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80.0,6.0,11962391.0,4.0,4.0,511.0,972.0,511.0,0.0,127.750000,255.500000,972.0,0.0,243.0,486.0,123.971872,0.668763,1.708913e+06,2.971775e+06,6957123.0,3.0,6959413.0,2.319804e+06,4.016290e+06,6957417.0,309.0,11962388.0,3.987463e+06,3.587107e+06,6957123.0,1998.0,0.0,0.0,0.0,0.0,136.0,136.0,0.334381,0.334381,0.0,972.0,164.777778,346.687820,120192.444444,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,185.375000,127.750000,243.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,511.0,4.0,972.0,26883.0,219.0,1.0,32.0,2001.0,0.0,2001.0,2001.0,6957123.0,0.0,6957123.0,6957123.0


In [48]:
# Here we can see the number of rows and columns for each table.
print('Training dataset shape: ', X_train.shape)
print('Testing dataset shape: ', X_test.shape)

Training dataset shape:  (60000, 78)
Testing dataset shape:  (15000, 78)


## Step 2. Normalization

The continuous feature values are normalized into the same feature space. This is important when using features that have different measurements, and is a general requirement of many machine learning algorithms. Therefore, the values for this dataset are also normalized using the Min-Max scaling technique, bringing them all within a range of [0,1].

In [49]:
from sklearn.preprocessing import MinMaxScaler

In [50]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_train

array([[6.76077833e-03, 3.52941176e-01, 5.34166680e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.22090805e-03, 3.52941176e-01, 7.75833353e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.22090805e-03, 3.52941176e-01, 1.04166669e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.22090805e-03, 3.52941176e-01, 2.50000006e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.63067531e-03, 1.00000000e+00, 7.45664144e-01, ...,
        3.84912719e-01, 3.51487659e-01, 6.02726213e-02],
       [1.22090805e-03, 3.52941176e-01, 4.17105427e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [51]:
X_train.shape

(60000, 78)

In [52]:
X_test = scaler.fit_transform(X_test)

In [53]:
X_test.shape

(15000, 78)

y_train and y_test have to be one-hot-encoded. That means they must have dimension (number_of_samples, 15), where 15 denotes number of classes.

In [54]:
y_vector = y_train.values.tolist()
print(len(y_vector))
y_test_v = y_test.values.tolist()
print(len(y_test_v))

60000
15000


In [55]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_vector, 15)
y_test = to_categorical(y_test_v, 15)
print(y_train.shape)
print(y_test.shape)

(60000, 15)
(15000, 15)


## Step 3. Build the model

In [56]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm

In [57]:
#hyper-params
batch_size = 128 # increasing batch size with more gpu added

input_dim = X_train.shape[1]
num_class = 15                   # 15 intrusion classes, including benign traffic class
num_epochs = 90

print(input_dim)
print(num_class)

78
15


In [58]:
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]
print(X_train_r.shape)

(60000, 78, 1)


In [59]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]
print(X_test_r.shape)

(15000, 78, 1)


**Model with 2 Con1D layers**

In [63]:
model2 = Sequential()

# input layer
model2.add(Conv1D(filters=16, kernel_size=3, padding='same', input_shape=(78,1)))
model2.add(BatchNormalization(axis=1))
model2.add(Activation('relu'))

model2.add(Conv1D(filters=32, kernel_size=3))
model2.add(BatchNormalization(axis=1))
model2.add(Activation('relu'))
 
model2.add(Flatten())
model2.add(Dropout(0.5))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(num_class))
model2.add(Activation('softmax'))

model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_14 (Conv1D)           (None, 78, 16)            64        
_________________________________________________________________
batch_normalization_14 (Batc (None, 78, 16)            312       
_________________________________________________________________
activation_21 (Activation)   (None, 78, 16)            0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 76, 32)            1568      
_________________________________________________________________
batch_normalization_15 (Batc (None, 76, 32)            304       
_________________________________________________________________
activation_22 (Activation)   (None, 76, 32)            0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 2432)             

In [64]:
learning_rates = 0.001
optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model2.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy']) 

In [65]:
model2.fit(X_train_r, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_r, y_test), verbose=1)

Epoch 1/90
469/469 [==============================] - 3s 5ms/step - loss: 0.8011 - accuracy: 0.7243 - val_loss: 0.4298 - val_accuracy: 0.8320
Epoch 2/90
469/469 [==============================] - 2s 4ms/step - loss: 0.3147 - accuracy: 0.8536 - val_loss: 0.3742 - val_accuracy: 0.8366
Epoch 3/90
469/469 [==============================] - 2s 4ms/step - loss: 0.2796 - accuracy: 0.8629 - val_loss: 0.4372 - val_accuracy: 0.8329
Epoch 4/90
469/469 [==============================] - 2s 4ms/step - loss: 0.2658 - accuracy: 0.8665 - val_loss: 0.4311 - val_accuracy: 0.8353
Epoch 5/90
469/469 [==============================] - 2s 4ms/step - loss: 0.2667 - accuracy: 0.8680 - val_loss: 0.4584 - val_accuracy: 0.8348
Epoch 6/90
469/469 [==============================] - 2s 4ms/step - loss: 0.2585 - accuracy: 0.8713 - val_loss: 0.4957 - val_accuracy: 0.8303
Epoch 7/90
469/469 [==============================] - 2s 4ms/step - loss: 0.2523 - accuracy: 0.8730 - val_loss: 0.4889 - val_accuracy: 0.8315
Epoch 

**Model with 5 Con1D layers**

In [17]:
model = Sequential()

# input layer
model.add(Conv1D(filters=64, kernel_size=23, padding='same', input_shape=(78,1)))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=13))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=256, kernel_size=7))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=512, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=256, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_class))
model.add(Activation('softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 78, 64)            1536      
_________________________________________________________________
batch_normalization (BatchNo (None, 78, 64)            312       
_________________________________________________________________
activation (Activation)      (None, 78, 64)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 39, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 27, 128)           106624    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 128)           108       
_________________________________________________________________
activation_1 (Activation)    (None, 27, 128)           0

In [18]:
learning_rates = 0.0001
optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy']) 

## Step 4. Training the model

In [20]:
# fit network
model.fit(X_train_r, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_r, y_test), verbose=1)

Epoch 1/90
30/30 [==============================] - 1s 44ms/step - loss: 0.2276 - accuracy: 0.8831 - val_loss: 0.2578 - val_accuracy: 0.8775
Epoch 2/90
30/30 [==============================] - 1s 40ms/step - loss: 0.2277 - accuracy: 0.8827 - val_loss: 0.2692 - val_accuracy: 0.8823
Epoch 3/90
30/30 [==============================] - 1s 40ms/step - loss: 0.2290 - accuracy: 0.8833 - val_loss: 0.2602 - val_accuracy: 0.8777
Epoch 4/90
30/30 [==============================] - 1s 41ms/step - loss: 0.2285 - accuracy: 0.8807 - val_loss: 0.2727 - val_accuracy: 0.8723
Epoch 5/90
30/30 [==============================] - 1s 40ms/step - loss: 0.2290 - accuracy: 0.8832 - val_loss: 0.2841 - val_accuracy: 0.8708
Epoch 6/90
30/30 [==============================] - 1s 40ms/step - loss: 0.2360 - accuracy: 0.8806 - val_loss: 0.2607 - val_accuracy: 0.8735
Epoch 7/90
30/30 [==============================] - 1s 41ms/step - loss: 0.2318 - accuracy: 0.8787 - val_loss: 0.2623 - val_accuracy: 0.8757
Epoch 8/90
30

In [108]:
# evaluate model
accuracy = model.evaluate(X_test_r, y_test, batch_size=batch_size, verbose=1)

15/15 [==============================] - 0s 10ms/step - loss: 0.7558 - accuracy: 0.7829


## Step 5. Reorder the features and put the most important feature first

In [32]:
column_names = [
 'Dst Port',
 'Protocol',
 'Flow Duration',
 'Tot Fwd Pkts',
 'Tot Bwd Pkts',
 'TotLen Fwd Pkts',
 'TotLen Bwd Pkts',
 'Fwd Pkt Len Max',
 'Fwd Pkt Len Min',
 'Fwd Pkt Len Mean',
 'Fwd Pkt Len Std',
 'Bwd Pkt Len Max',
 'Bwd Pkt Len Min',
 'Bwd Pkt Len Mean',
 'Bwd Pkt Len Std',
 'Flow Byts/s',
 'Flow Pkts/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Flow IAT Min',
 'Fwd IAT Tot',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Fwd IAT Min',
 'Bwd IAT Tot',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'Fwd Header Len',
 'Bwd Header Len',
 'Fwd Pkts/s',
 'Bwd Pkts/s',
 'Pkt Len Min',
 'Pkt Len Max',
 'Pkt Len Mean',
 'Pkt Len Std',
 'Pkt Len Var',
 'FIN Flag Cnt',
 'SYN Flag Cnt',
 'RST Flag Cnt',
 'PSH Flag Cnt',
 'ACK Flag Cnt',
 'URG Flag Cnt',
 'CWE Flag Count',
 'ECE Flag Cnt',
 'Down/Up Ratio',
 'Pkt Size Avg',
 'Fwd Seg Size Avg',
 'Bwd Seg Size Avg',
 'Fwd Byts/b Avg',
 'Fwd Pkts/b Avg',
 'Fwd Blk Rate Avg',
 'Bwd Byts/b Avg',
 'Bwd Pkts/b Avg',
 'Bwd Blk Rate Avg',
 'Subflow Fwd Pkts',
 'Subflow Fwd Byts',
 'Subflow Bwd Pkts',
 'Subflow Bwd Byts',
 'Init Fwd Win Byts',
 'Init Bwd Win Byts',
 'Fwd Act Data Pkts',
 'Fwd Seg Size Min',
 'Active Mean',
 'Active Std',
 'Active Max',
 'Active Min',
 'Idle Mean',
 'Idle Std',
 'Idle Max',
 'Idle Min']

In [33]:
len(column_names)

78

According to "**Selection and Performance Analysis of CICIDS2017 Features Importance**", the important features are: *Destination Port, Fwd IAT Min, Init_Win_bytes_Forward, Init_Win_bytes_backward* and *FlowIATMin*

In [34]:
important_column_names = ['Dst Port',
 'Fwd IAT Min',
 'Init Fwd Win Byts',
 'Init Bwd Win Byts',
 'Flow IAT Min',
 'Flow Duration',
 'Tot Fwd Pkts',
 'Tot Bwd Pkts',
 'TotLen Fwd Pkts',
 'TotLen Bwd Pkts',
 'Fwd Pkt Len Max',
 'Fwd Pkt Len Min',
 'Fwd Pkt Len Mean',
 'Fwd Pkt Len Std',
 'Bwd Pkt Len Max',
 'Bwd Pkt Len Min',
 'Bwd Pkt Len Mean',
 'Bwd Pkt Len Std',
 'Protocol',
 'Flow Byts/s',
 'Flow Pkts/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Fwd IAT Tot',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Bwd IAT Tot',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'Fwd Header Len',
 'Bwd Header Len',
 'Fwd Pkts/s',
 'Bwd Pkts/s',
 'Pkt Len Min',
 'Pkt Len Max',
 'Pkt Len Mean',
 'Pkt Len Std',
 'Pkt Len Var',
 'FIN Flag Cnt',
 'SYN Flag Cnt',
 'RST Flag Cnt',
 'PSH Flag Cnt',
 'ACK Flag Cnt',
 'URG Flag Cnt',
 'CWE Flag Count',
 'ECE Flag Cnt',
 'Down/Up Ratio',
 'Pkt Size Avg',
 'Fwd Seg Size Avg',
 'Bwd Seg Size Avg',
 'Fwd Byts/b Avg',
 'Fwd Pkts/b Avg',
 'Fwd Blk Rate Avg',
 'Bwd Byts/b Avg',
 'Bwd Pkts/b Avg',
 'Bwd Blk Rate Avg',
 'Subflow Fwd Pkts',
 'Subflow Fwd Byts',
 'Subflow Bwd Pkts',
 'Subflow Bwd Byts',
 'Fwd Act Data Pkts',
 'Fwd Seg Size Min',
 'Active Mean',
 'Active Std',
 'Active Max',
 'Active Min',
 'Idle Mean',
 'Idle Std',
 'Idle Max',
 'Idle Min']

Read X_train and X_test datasets.

In [35]:
X_train = pd.read_csv(cleanfile, skiprows=0,index_col=0) 
X_test = pd.read_csv(xtestsmall, skiprows=0, index_col=0)
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77
0,80.0,6.0,69475.0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,43.181000,34737.500000,1.095238e+04,42482.0,26993.0,69475.0,3.473750e+04,1.095238e+04,42482.0,26993.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,96.0,0.0,43.181000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,225.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,53102.0,6.0,51.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,39215.686270,51.000000,0.000000e+00,51.0,51.0,51.0,5.100000e+01,0.000000e+00,51.0,51.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,39215.686270,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,256.0,-1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8080.0,6.0,517.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,3868.471954,517.000000,0.000000e+00,517.0,517.0,517.0,5.170000e+02,0.000000e+00,517.0,517.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,3868.471954,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2052.0,-1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22.0,6.0,369481.0,22.0,22.0,1928.0,2665.0,640.0,0.0,87.636364,137.780552,976.0,0.0,121.136364,258.641560,12430.950441,119.085961,8592.581395,2.276687e+04,97771.0,2.0,369439.0,1.759233e+04,3.575191e+04,133959.0,163.0,369470.0,17593.809524,38826.475848,129556.0,11.0,0.0,0.0,0.0,0.0,712.0,712.0,59.542981,59.542981,0.0,976.0,102.066667,203.740967,41510.381818,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,104.386364,87.636364,121.136364,0.0,0.0,0.0,0.0,0.0,0.0,22.0,1928.0,22.0,2665.0,26883.0,230.0,16.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80.0,6.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,16.0,5007525.0,1.251881e+06,2.467885e+06,4953524.0,406.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,8192.0,221.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Add the column names to them

In [36]:
 X_train.columns =  column_names
 X_test.columns = column_names
 X_train.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,80.0,6.0,69475.0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,43.181000,34737.500000,1.095238e+04,42482.0,26993.0,69475.0,3.473750e+04,1.095238e+04,42482.0,26993.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,96.0,0.0,43.181000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,225.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,53102.0,6.0,51.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,39215.686270,51.000000,0.000000e+00,51.0,51.0,51.0,5.100000e+01,0.000000e+00,51.0,51.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,39215.686270,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,256.0,-1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8080.0,6.0,517.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,3868.471954,517.000000,0.000000e+00,517.0,517.0,517.0,5.170000e+02,0.000000e+00,517.0,517.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,3868.471954,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2052.0,-1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22.0,6.0,369481.0,22.0,22.0,1928.0,2665.0,640.0,0.0,87.636364,137.780552,976.0,0.0,121.136364,258.641560,12430.950441,119.085961,8592.581395,2.276687e+04,97771.0,2.0,369439.0,1.759233e+04,3.575191e+04,133959.0,163.0,369470.0,17593.809524,38826.475848,129556.0,11.0,0.0,0.0,0.0,0.0,712.0,712.0,59.542981,59.542981,0.0,976.0,102.066667,203.740967,41510.381818,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,104.386364,87.636364,121.136364,0.0,0.0,0.0,0.0,0.0,0.0,22.0,1928.0,22.0,2665.0,26883.0,230.0,16.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80.0,6.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,16.0,5007525.0,1.251881e+06,2.467885e+06,4953524.0,406.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,8192.0,221.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


reorder the columns

In [37]:
X_train_important = X_train.reindex(important_column_names, axis=1)
X_test_important = X_test.reindex(important_column_names, axis=1)
X_train_important.head()

,Dst Port,Fwd IAT Min,Init Fwd Win Byts,Init Bwd Win Byts,Flow IAT Min,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Protocol,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,80.0,26993.0,225.0,-1.0,26993.0,69475.0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,6.0,0.000000,43.181000,34737.500000,1.095238e+04,42482.0,69475.0,3.473750e+04,1.095238e+04,42482.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,96.0,0.0,43.181000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,53102.0,51.0,256.0,-1.0,51.0,51.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,6.0,0.000000,39215.686270,51.000000,0.000000e+00,51.0,51.0,5.100000e+01,0.000000e+00,51.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,39215.686270,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8080.0,517.0,2052.0,-1.0,517.0,517.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,6.0,0.000000,3868.471954,517.000000,0.000000e+00,517.0,517.0,5.170000e+02,0.000000e+00,517.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,3868.471954,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22.0,163.0,26883.0,230.0,2.0,369481.0,22.0,22.0,1928.0,2665.0,640.0,0.0,87.636364,137.780552,976.0,0.0,121.136364,258.641560,6.0,12430.950441,119.085961,8592.581395,2.276687e+04,97771.0,369439.0,1.759233e+04,3.575191e+04,133959.0,369470.0,17593.809524,38826.475848,129556.0,11.0,0.0,0.0,0.0,0.0,712.0,712.0,59.542981,59.542981,0.0,976.0,102.066667,203.740967,41510.381818,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,104.386364,87.636364,121.136364,0.0,0.0,0.0,0.0,0.0,0.0,22.0,1928.0,22.0,2665.0,16.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80.0,406.0,8192.0,221.0,16.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,6.0,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,5007525.0,1.251881e+06,2.467885e+06,4953524.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
X_test_important.head()

,Dst Port,Fwd IAT Min,Init Fwd Win Byts,Init Bwd Win Byts,Flow IAT Min,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Protocol,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,80.0,53422.0,225.0,-1.0,53422.0,53422.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,6.0,0.000000e+00,37.437760,5.342200e+04,0.000000e+00,53422.0,53422.0,53422.0,0.000000,53422.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,37.437760,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,51863.0,1.0,946.0,-1.0,1.0,3.0,3.0,0.0,31.0,0.0,31.0,0.0,10.333333,17.897858,0.0,0.0,0.0,0.0,6.0,1.030000e+07,1000000.000000,1.500000e+00,7.071068e-01,2.0,3.0,1.5,0.707107,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1000000.000000,0.000000,0.0,31.0,7.75,15.500000,240.250000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.333333,10.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,31.0,0.0,0.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,80.0,81829392.0,211.0,219.0,2.0,81829392.0,2.0,1.0,8.0,0.0,8.0,0.0,4.000000,5.656854,0.0,0.0,0.0,0.0,6.0,9.776438e-02,0.036662,4.091470e+07,5.786212e+07,81829390.0,81829392.0,81829392.0,0.000000,81829392.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,64.0,32.0,0.024441,0.012221,0.0,8.0,4.00,4.618802,21.333333,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.333333,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,1.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,53.0,18491.0,-1.0,-1.0,33.0,101932.0,2.0,2.0,92.0,296.0,46.0,46.0,46.000000,0.000000,148.0,148.0,148.0,0.0,17.0,3.806459e+03,39.241848,3.397733e+04,4.379175e+04,83408.0,18491.0,18491.0,0.000000,18491.0,33.0,33.0,0.0,33.0,33.0,0.0,0.0,0.0,0.0,16.0,16.0,19.620924,19.620924,46.0,148.0,86.80,55.867701,3121.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,108.500000,46.000000,148.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,92.0,2.0,296.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80.0,100.0,2047.0,-1.0,100.0,100.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,6.0,0.000000e+00,20000.000000,1.000000e+02,0.000000e+00,100.0,100.0,100.0,0.000000,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,20000.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 6. Normalization Again

In [39]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_important)
X_test = scaler.fit_transform(X_test_important)

In [40]:
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]

In [41]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]

## Step 7. Reuse the Model and train it again

In [46]:
model = Sequential()

# input layer
model.add(Conv1D(filters=32, kernel_size=17, padding='same', input_shape=(78,1)))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=7))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=256, kernel_size=5))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=512, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=256, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_class))
model.add(Activation('softmax'))


model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_15 (Conv1D)           (None, 78, 32)            576       
_________________________________________________________________
batch_normalization_15 (Batc (None, 78, 32)            312       
_________________________________________________________________
activation_18 (Activation)   (None, 78, 32)            0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 39, 32)            0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 33, 128)           28800     
_________________________________________________________________
batch_normalization_16 (Batc (None, 33, 128)           132       
_________________________________________________________________
activation_19 (Activation)   (None, 33, 128)          

In [42]:
#num_epochs = 35

model2.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy']) 

In [43]:
model2.fit(X_train_r, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_r, y_test), verbose=1)

Epoch 1/90
235/235 [==============================] - 2s 5ms/step - loss: 1.1727 - accuracy: 0.6958 - val_loss: 0.3491 - val_accuracy: 0.8605
Epoch 2/90
235/235 [==============================] - 1s 4ms/step - loss: 0.3360 - accuracy: 0.8449 - val_loss: 0.3285 - val_accuracy: 0.8611
Epoch 3/90
235/235 [==============================] - 1s 4ms/step - loss: 0.3000 - accuracy: 0.8583 - val_loss: 0.3139 - val_accuracy: 0.8661
Epoch 4/90
235/235 [==============================] - 1s 5ms/step - loss: 0.2900 - accuracy: 0.8577 - val_loss: 0.3088 - val_accuracy: 0.8672
Epoch 5/90
235/235 [==============================] - 1s 4ms/step - loss: 0.2725 - accuracy: 0.8687 - val_loss: 0.3158 - val_accuracy: 0.8571
Epoch 6/90
235/235 [==============================] - 1s 4ms/step - loss: 0.2715 - accuracy: 0.8662 - val_loss: 0.2957 - val_accuracy: 0.8693
Epoch 7/90
235/235 [==============================] - 1s 4ms/step - loss: 0.2669 - accuracy: 0.8676 - val_loss: 0.2915 - val_accuracy: 0.8681
Epoch 

In [44]:
learning_rates = 1e-6

optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model2.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy']) 

In [45]:
model2.fit(X_train_r, y_train, epochs=60, batch_size=batch_size, validation_data=(X_test_r, y_test), verbose=1)

Epoch 1/60
235/235 [==============================] - 2s 5ms/step - loss: 0.2195 - accuracy: 0.8857 - val_loss: 0.2878 - val_accuracy: 0.8759
Epoch 2/60
235/235 [==============================] - 1s 4ms/step - loss: 0.2178 - accuracy: 0.8849 - val_loss: 0.2871 - val_accuracy: 0.8757
Epoch 3/60
235/235 [==============================] - 1s 4ms/step - loss: 0.2164 - accuracy: 0.8884 - val_loss: 0.2866 - val_accuracy: 0.8799
Epoch 4/60
235/235 [==============================] - 1s 5ms/step - loss: 0.2176 - accuracy: 0.8870 - val_loss: 0.2867 - val_accuracy: 0.8797
Epoch 5/60
235/235 [==============================] - 1s 4ms/step - loss: 0.2148 - accuracy: 0.8883 - val_loss: 0.2864 - val_accuracy: 0.8799
Epoch 6/60
235/235 [==============================] - 1s 4ms/step - loss: 0.2167 - accuracy: 0.8874 - val_loss: 0.2866 - val_accuracy: 0.8800
Epoch 7/60
235/235 [==============================] - 1s 4ms/step - loss: 0.2165 - accuracy: 0.8877 - val_loss: 0.2863 - val_accuracy: 0.8800
Epoch 